In [2]:
%serialconnect to port=/dev/tty.usbserial-0001 --baud=115200

Found serial ports: /dev/cu.usbserial-0001, /dev/cu.Bluetooth-Incoming-Port, /dev/cu.wlan-debug 
Connecting to --port=/dev/cu.usbserial-0001 --baud=115200 
Ready.


In [7]:
# Created: 2021-12-19 11:06:43
# License: MIT, Copyright (c) 2021 Olivier Lenoir
# Language: MicroPython v1.17
# Project: Roll Pitch Yaw detector
# Description: GY-521 MPU-6050 3-axis gyroscope and acceleration sensor
# Detect if Roll Pitch Yaw is greater than a trigger point


from machine import Pin, SoftI2C
from ustruct import unpack
from utime import sleep_ms

MPU6050_ADDR = const(0x68)
MPU6050_TEMP_OUT = const(0x41)
MPU6050_ACCEL_OUT = const(0x3b)
MPU6050_GYRO_OUT = const(0x43)


class RPY(object):
    def __init__(self, ic2, addr=MPU6050_ADDR):
        self.i2c = i2c
        self.addr = addr
        self.mean = [0, 0, 0]
        self.stddev = [1, 1, 1]
        self._init()

    def _init(self):
        # Wake sensor
        self.i2c.writeto_mem(MPU6050_ADDR, 0x6b, bytes([0]))
        sleep_ms(50)

    def gyro_raw(self):
        #Read and unpack gyro raw data
        return unpack('>hhh', self.i2c.readfrom_mem(self.addr, MPU6050_GYRO_OUT, 6))

    def gyro(self):
        # Read and convert gyro to °/s
        return [a / 131 for a in self.gyro_raw()]

    def gyro_mean_sigma(self, rate=20, p=100):
        # Calculate sensor mean and standard deviation, sensor need to be static
        s = [0, 0, 0]
        sq = [0, 0, 0]
        for _ in range(p):
            d = self.gyro_raw()
            s = [a + b for a, b in zip(s, d)]
            sq = [a + b * b for a, b in zip(sq, d)]
            sleep_ms(rate)
        self.mean = [_s / p for _s in s]
        self.stddev = [(_sq / p - (_s / p) ** 2) ** 0.5 for _s, _sq in zip(s, sq)]

    def gyro_raw_std_score(self):
        # Standardize gyro raw with standard score
        return [(x - m) / p for x, m, p in zip(detector.gyro_raw(), self.mean, self.stddev)]


if __name__ == '__main__':
    i2c = SoftI2C(scl=Pin(23), sda=Pin(22))
    detector = RPY(i2c)
    detector.gyro_mean_sigma()

    while True:
        m = [0, 0, 0]
        print('Reset')
        for _ in range(200):
            # cumulative gyro
            m = [sum(a) for a in zip(m, detector.gyro_raw_std_score())]
            print(m)
            # trigger point 1000
            if any(a > 1000 for a in map(abs, m)):
                print('You move!')
            sleep_ms(100)

Reset
[0.1666671, -0.650142, -1.192484]
[0.8658163, 0.580863, -0.05703318]
[1.431845, 0.06836593, 0.1472436]
[1.784881, 0.4735016, 1.224496]
[1.232697, 2.668021, 1.952558]
[2.091591, 2.797867, 3.553596]
[2.710868, 3.891227, 2.710302]
[2.211932, 5.993983, 2.623587]
[1.074019, 7.959095, 2.711467]
[0.9478206, 8.36423, 2.042768]
[0.7683744, 8.815248, 2.538037]
[1.334403, 9.95449, 2.043933]
[1.634191, 8.478478, 3.819566]
[-3.710331, 14.20588, 24.62606]
[13.28277, -330.7401, 192.1506]
[-12.40257, -1252.648, -155.2639]
You move!
[-167.3746, -2108.119, -139.8117]
You move!
[-205.2271, -3424.975, -499.0405]
You move!
[-243.1063, -4323.483, -713.6464]
You move!
[-234.766, -4790.061, -827.6273]
You move!
[-269.0243, -4935.101, -753.5111]
You move!
[-276.2592, -5010.355, -772.8615]
You move!
[-271.7794, -4995.176, -788.8946]
You move!
[-250.7394, -4980.547, -780.3679]
You move!
[-244.6622, -4952.935, -810.6013]
You move!
[-242.4987, -4958.356, -826.8672]
You move!
[-243.4769, -4969.192, -800.3572]

Traceback (most recent call last):
  File "<stdin>", line 73, in <module>
KeyboardInterrupt: 
